In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
## bUILT-iN pACKAGES
import sys, os, time, bz2, zlib, pickle, math, json, csv
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)
%matplotlib inline
from IPython.display import display

## lAST sUMMER
from romtoslp import *
from sentences import *
from DCS import *
import MatDB
from heap_n_PrimMST import *
from ECL_MST import *

## lAST yEAR
# from word_definite import *
from nnet import *
# from heap_n_PrimMST import *
# from word_definite import *

In [3]:
# from MatDB import *
import word_definite as WD
from heap_n_PrimMST import *
from nnet import *

In [4]:
import winsound
def playBeep():
    for i in range(3):
        winsound.Beep(2200, 300)
        winsound.Beep(2600, 300)

In [5]:
loaded_SKT = pickle.load(open('../Simultaneous_CompatSKT_10K.p', 'rb'), encoding=u'utf-8')
loaded_DCS = pickle.load(open('../Simultaneous_DCS_10K.p', 'rb'), encoding=u'utf-8')

In [6]:
# dataset_4k_1k = pickle.load(open('../SmallDataset_4K_1K.p', 'rb'))
# TrainFiles = dataset_4k_1k['TrainFiles']
# TestFiles = dataset_4k_1k['TestFiles']

# dataset_6k_3k = pickle.load(open('../SmallDataset_6K_3K.p', 'rb'))
# TrainFiles_2 = dataset_6k_3k['TrainFiles']
# TestFiles_2 = dataset_6k_3k['TestFiles']

In [7]:
# matDB = MatDB.MatDB()

In [8]:
# Create Training File List
excluded_files = []
with open('inputs/Baseline4_advSample.csv', 'r') as f_handle:
    opener = csv.reader(f_handle)
    for line in opener:
        excluded_files.append(line[1].replace('.p', '.ds.bz2'))

bz2_input_folder = '../NewData/skt_dcs_DS.bz2_1L_bigram_10K//'
# bz2_input_folder = '/home/rs/15CS91R05/vishnu/Data/skt_dcs_DS.bz2_compat_10k_check_again/'
all_files = []
skipped = 0
for f in os.listdir(bz2_input_folder):
    if '.ds.bz2' in f:
        if f in excluded_files:
            skipped += 1
            continue
        all_files.append(f)

print(skipped, 'files will not be used for training')
print('Size of training set:', len(all_files))

TrainFiles = all_files        

957 files will not be used for training
Size of training set: 8219


In [9]:
def open_dsbz2(filename):
    with bz2.BZ2File(filename, 'r') as f:
        loader = pickle.load(f)
    
    conflicts_Dict_correct = loader['conflicts_Dict_correct']
    nodelist_to_correct_mapping = loader['nodelist_to_correct_mapping']
    nodelist_correct = loader['nodelist_correct']
    featVMat_correct = loader['featVMat_correct']
    featVMat = loader['featVMat']
    conflicts_Dict = loader['conflicts_Dict']
    nodelist = loader['nodelist']
    
    return (nodelist_correct, conflicts_Dict_correct, featVMat_correct, nodelist_to_correct_mapping,\
            nodelist, conflicts_Dict, featVMat)


In [10]:
"""
################################################################################################
######################  CREATE SEVERAL DATA STRUCTURES FROM SENTENCE/DCS  ######################
###########################  NODELIST, ADJACENCY LIST, GRAPH, HEAP #############################
################################################################################################
"""
def GetTrainingKit(sentenceObj, dcsObj):
    nodelist = GetNodes(sentenceObj)
    
    # Nodelist with only the correct_nodes
    nodelist2 = GetNodes(sentenceObj)
    nodelist2_to_correct_mapping = {}
    nodelist_correct = []
    search_key = 0
    first_key = 0
    for chunk_id in range(len(dcsObj.lemmas)):
        while nodelist2[first_key].chunk_id != chunk_id:
            first_key += 1
        for j in range(len(dcsObj.lemmas[chunk_id])):
            search_key = first_key
            while (nodelist2[search_key].lemma != rom_slp(dcsObj.lemmas[chunk_id][j])) or (nodelist2[search_key].cng != dcsObj.cng[chunk_id][j]):
                search_key += 1
                if search_key >= len(nodelist2) or nodelist2[search_key].chunk_id > chunk_id:
                    break
    #         print((rom_slp(dcsObj.lemmas[chunk_id][j]), dcsObj.cng[chunk_id][j]))
    #         print(nodelist[search_key])
            nodelist2_to_correct_mapping[len(nodelist_correct)] = search_key
            nodelist_correct.append(nodelist2[search_key])
    return (nodelist, nodelist_correct, nodelist2_to_correct_mapping)
    

def GetGraph(nodelist, neuralnet):
    if not neuralnet.outer_relu:
        conflicts_Dict = Get_Conflicts(nodelist)

        featVMat = Get_Feat_Vec_Matrix(nodelist, conflicts_Dict)

        (WScalarMat, SigmoidGateOutput) = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        return (conflicts_Dict, featVMat, WScalarMat, SigmoidGateOutput)
    else:
        conflicts_Dict = Get_Conflicts(nodelist)

        featVMat = Get_Feat_Vec_Matrix(nodelist, conflicts_Dict)

        WScalarMat = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        return (conflicts_Dict, featVMat, WScalarMat)

In [11]:
trainingStatus = defaultdict(lambda: bool(False))

In [12]:
"""
################################################################################################
##############################  TRAIN FUNCTION  ################################################
################################################################################################
"""

def train_generator(loaded_SKT, loaded_DCS, bz2_input_folder, n_trainset = -1, iterationPerBatch = 10, filePerBatch = 20, _debug = True):
    # Train
    if n_trainset == -1:
        n_trainset = len(TrainFiles)
        totalBatchToTrain = math.ceil(n_trainset/filePerBatch)
    else:
        totalBatchToTrain = math.ceil(n_trainset/filePerBatch)
    
    register_nnet(trainer.neuralnet, bz2_input_folder)
    for iterout in range(totalBatchToTrain):
        # Add timer
        startT = time.time()

        # Change current batch
        if(iterout % 50 == 0):
            trainer.Save(p_name.replace('.p', '_i{}.p'.format(iterout)))
        else:
            trainer.Save(p_name)
        print('Batch: ', iterout)
        files_for_batch = TrainFiles[iterout*filePerBatch:(iterout + 1)*filePerBatch]
        print(files_for_batch)
        # trainer.Load('outputs/neuralnet_trained.p')
        try:
            # Run few times on same set of files
            for iterin in range(iterationPerBatch):
                print('ITERATION IN', iterin)        
                for fn in files_for_batch:
                    trainFileName = fn.replace('.ds.bz2', '.p2')
                    sentenceObj = loaded_SKT[trainFileName]
                    dcsObj = loaded_DCS[trainFileName]
                    if trainingStatus[sentenceObj.sent_id]:
                        continue
                    # trainer.Save('outputs/saved_trainer.p')
                    try:
                        trainer.Train(sentenceObj, dcsObj, bz2_input_folder, _debug= _debug)
                    except (IndexError, KeyError) as e:
                        print('\x1b[31mFailed: {} \x1b[0m'.format(sentenceObj.sent_id))
                sys.stdout.flush() # Flush IO buffer 
            finishT = time.time()
            print('Avg. time taken by 1 file(1 iteration): {:.3f}'.format((finishT - startT)/(iterationPerBatch*filePerBatch)))
        except KeyboardInterrupt:
            print('Training paused')
            trainer.Save(p_name)
            yield None
    trainer.Save(p_name)
                
def test(loaded_SKT, loaded_DCS, n_testSet = -1, _testFiles = None, n_checkpt = 100):
    total_lemma = 0;
    correct_lemma = 0;

    total_word = 0;
    total_output_nodes = 0
    correct_word = 0;
    file_counter = 0
    if _testFiles is None:
        if n_testSet == -1:
            _testFiles = TestFiles
        else:
            _testFiles = TestFiles[0:n_testSet]
    else:
        if n_testSet == -1:
            _testFiles = _testFiles
        else:
            _testFiles = _testFiles[0:n_testSet]
            
    recalls = []
    recalls_of_word = []
    precisions = []
    precisions_of_words = []
    for fn in _testFiles:
        if file_counter % n_checkpt == 0:
            print(file_counter,' Checkpoint... ')
            sys.stdout.flush() # Flush IO buffer 
        
        file_counter += 1
        
        testFileName = fn.replace('.ds.bz2', '.p2')
        sentenceObj = loaded_SKT[testFileName]
        dcsObj = loaded_DCS[testFileName]
        
        try:
            (word_match, lemma_match, n_dcsWords, n_output_nodes) = trainer.Test(sentenceObj, dcsObj)
            
            recalls.append(lemma_match/n_dcsWords)
            recalls_of_word.append(word_match/n_dcsWords)
            
            precisions.append(lemma_match/n_output_nodes)
            precisions_of_words.append(word_match/n_output_nodes)
            
            total_lemma += n_dcsWords
            total_word += n_dcsWords
            
            total_output_nodes += n_output_nodes            
            
            correct_lemma += lemma_match
            correct_word += word_match
        except (IndexError, KeyError) as e:
            print('Failed!')        

    print('Avg. Micro Recall of Lemmas: {}'.format(np.mean(np.array(recalls))))
    print('Avg. Micro Recall of Words: {}'.format(np.mean(np.array(recalls_of_word))))
    print('Avg. Micro Precision of Lemmas: {}'.format(np.mean(np.array(precisions))))
    print('Avg. Micro Precision of Words: {}'.format(np.mean(np.array(precisions_of_words))))
    
    return (recalls, recalls_of_word, precisions, precisions_of_words)
    

In [13]:
# NOW-OLD FUNCTION
# def GetLoss(_mst_adj_graph, _mask_de_correct_edges, _negLogLikelies):
#     _negLogLikelies = _negLogLikelies.copy()
#     _negLogLikelies[~_mst_adj_graph] = 0
#     _negLogLikelies[~_mask_de_correct_edges] *= -1 # BAKA!!! Check before you try to fix this again
#     return np.sum(_negLogLikelies)    

# NEW FUNCTION
def GetLoss(_mst_adj_graph, _mask_de_correct_edges, _WScalarMat):
    _WScalarMat = _WScalarMat.copy()
    _WScalarMat[_mst_adj_graph&(~_mask_de_correct_edges)] *= -1 # BAKA!!! Check before you try to fix this again
    _WScalarMat[~_mst_adj_graph] = 0
    return np.sum(_WScalarMat)

In [14]:
"""
################################################################################################
##############################  GET A FILENAME TO SAVE WEIGHTS  ################################
################################################################################################
"""
import time
st = str(int((time.time() * 1e6) % 1e13))
log_name = 'logs/train_nnet_t{}.out'.format(st)
odir = 'outputs/train_t{}'.format(st)
p_name = 'outputs/train_t{}/nnet.p'.format(st)
print('nEURAL nET wILL bE sAVED hERE: ', p_name)

nEURAL nET wILL bE sAVED hERE:  outputs/train_t1711879920150/nnet.p


In [15]:
def register_nnet(nnet, bz2_input_folder):
    if not os.path.isdir(odir):
        os.mkdir(odir)
    if not os.path.isfile('outputs/nnet_LOGS.csv'):
        with open('outputs/nnet_LOGS.csv', 'a') as fh:
            csv_r = csv.writer(fh)
            csv_r.writerow(['odir', 'p_name', 'hidden_layer_size', '_edge_vector_dim'])
    with open('outputs/nnet_LOGS.csv', 'a') as fh:
        csv_r = csv.writer(fh)
        csv_r.writerow([odir, p_name, nnet.n, nnet.d, bz2_input_folder])

In [40]:
class Trainer:
    def __init__(self, modelFile = None):
        if modelFile is None:
            self.hidden_layer_size = 1200
            self._edge_vector_dim = 1500
            # self._edge_vector_dim = WD._edge_vector_dim
            # self._full_cnglist = list(WD.mat_cngCount_1D)
            
            self.neuralnet = NN(self._edge_vector_dim, self.hidden_layer_size, outer_relu=True)
            self.history = defaultdict(lambda: list())
        else:
            loader = pickle.load(open(filename, 'rb'))
            
            self.neuralnet.n = loader['n']
            self.neuralnet.d = loader['d']

            self.neuralnet = NN(self._edge_vector_dim, self.hidden_layer_size, outer_relu=True)

            self.neuralnet.U = loader['U']
            self.neuralnet.W = loader['W']
            self.neuralnet.B1 = loader['B1']
            self.neuralnet.B2 = loader['B2']
            
            self.history = defaultdict(lambda: list())
            
        # SET LEARNING RATES
        self.neuralnet.etaW = 3e-4
        self.neuralnet.etaB1 = 1e-4
        
        self.neuralnet.etaU = 1e-4
        self.neuralnet.etaB2 = 1e-4
            
    def Reset(self):
        self.neuralnet = NN(self._edge_vector_dim, self.hidden_layer_size)
        self.history = defaultdict(lambda: list())
        
    def Save(self, filename):
        print('Weights Saved: ', filename)
        pickle.dump({
                'U': self.neuralnet.U,
                'W': self.neuralnet.W,
                'n': self.neuralnet.n,
                'd': self.neuralnet.d,
                'B1': self.neuralnet.B1,
                'B2': self.neuralnet.B2
            }, open(filename, 'wb'))
        return
        
    
    def Load(self, filename):
        loader = pickle.load(open(filename, 'rb'))
        self.neuralnet.U = loader['U']
        self.neuralnet.W = loader['W']
        self.neuralnet.B1 = loader['B1']
        self.neuralnet.B2 = loader['B2']
        self.neuralnet.hidden_layer_size = loader['n']
        self.neuralnet._edge_vector_dim = loader['d']
        
    def Test(self, sentenceObj, dcsObj, dsbz2_name, _dump = False, _outFile = None):
        if _dump:
            if _outFile is None:
                raise Exception('WTH r u thinking! pass me outFolder')
        neuralnet = self.neuralnet
        minScore = np.inf
        minMst = None
        
        # dsbz2_name = sentenceObj.sent_id + '.ds.bz2'
        (nodelist_correct, conflicts_Dict_correct, featVMat_correct, nodelist_to_correct_mapping,\
            nodelist, conflicts_Dict, featVMat) = open_dsbz2(dsbz2_name)
        
        # if len(nodelist) > 50:
        #     return None

        if not self.neuralnet.outer_relu:
            (WScalarMat, SigmoidGateOutput) = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        else:
            WScalarMat = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        
        # print('NeuralNet Time: ', time.time() - startT)
        # startT = time.time()
        
        # Get all MST
        for source in range(len(nodelist)):
            (mst_nodes, mst_adj_graph, _) = MST(nodelist, WScalarMat, conflicts_Dict, source)
            # print('.', end = '')
            score = GetMSTWeight(mst_adj_graph, WScalarMat)
            if(score < minScore):
                minScore = score
                minMst = mst_nodes
        dcsLemmas = [[rom_slp(l) for l in arr]for arr in dcsObj.lemmas]
        word_match = 0
        lemma_match = 0
        n_output_nodes = 0
        
        if _dump:
            predicted_lemmas = [sentenceObj.sent_id]
            predicted_cngs = [sentenceObj.sent_id]
        
        for chunk_id, wdSplit in minMst.items():
            for wd in wdSplit:
                if _dump:
                    predicted_lemmas.append(wd.lemma)
                    predicted_cngs.append(wd.cng)
                n_output_nodes += 1
                # Match lemma
                search_result = [i for i, j in enumerate(dcsLemmas[chunk_id]) if j == wd.lemma]
                if len(search_result) > 0:
                    lemma_match += 1
                # Match CNG
                for i in search_result:
                    if(dcsObj.cng[chunk_id][i] == str(wd.cng)):
                        word_match += 1
                        # print(wd.lemma, wd.cng)
                        break
        dcsLemmas = [l for arr in dcsObj.lemmas for l in arr]
        
        if _dump:
            with open(_outFile, 'a') as fh:
                dcsv = csv.writer(fh)
                dcsv.writerow(predicted_lemmas)
                dcsv.writerow(predicted_cngs)
                dcsv.writerow([sentenceObj.sent_id, word_match, lemma_match, len(dcsLemmas), n_output_nodes])
        
        # print('All MST Time: ', time.time() - startT)
        # print('Node Count: ', len(nodelist))
#         print('\nFull Match: {}, Partial Match: {}, OutOf {}, NodeCount: {}, '.\
#               format(word_match, lemma_match, len(dcsLemmas), len(nodelist)))
        return (word_match, lemma_match, len(dcsLemmas), n_output_nodes)
    
    def Train(self, sentenceObj, dcsObj, bz2_input_folder, _debug = True):
        # Hyperparameter for hinge loss: m
        m_hinge_param = 14
        
        dsbz2_name = sentenceObj.sent_id + '.ds.bz2'
        (nodelist_correct, conflicts_Dict_correct, featVMat_correct, nodelist_to_correct_mapping,\
            nodelist, conflicts_Dict, featVMat) = open_dsbz2(bz2_input_folder + dsbz2_name)
        # Train for large graphs separately
#         if len(nodelist) < 40:
#             return
        
        """ FORM MAXIMUM(ENERGY) SPANNING TREE OF THE GOLDEN GRAPH : WORST GOLD STRUCTURE """
        WScalarMat_correct = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat_correct, nodelist_correct,\
                                                                      conflicts_Dict_correct, self.neuralnet)
        source = 0
        """ Find the max spanning tree : negative Weight matrix passed """
#         (max_st_gold_ndict, max_st_adj_gold_small, _) =\
#             MST(nodelist_correct, -WScalarMat_correct, conflicts_Dict_correct, source)
        (max_st_gold_ndict, max_st_adj_gold_small, _) =\
            MST(nodelist_correct, -WScalarMat_correct, conflicts_Dict_correct, source)
        energy_gold_max_ST = np.sum(WScalarMat_correct[max_st_adj_gold_small])
        
        """ Convert correct spanning tree graph adj matrix to full marix dimensions """
        """ Create full-size adjacency matrix for correct_mst_small """
        nodelen = len(nodelist)
        max_st_adj_gold = np.ndarray((nodelen, nodelen), np.bool)*False # T_STAR
        for i in range(max_st_adj_gold_small.shape[0]):
            for j in range(max_st_adj_gold_small.shape[1]):
                max_st_adj_gold[nodelist_to_correct_mapping[i], nodelist_to_correct_mapping[j]] =\
                    max_st_adj_gold_small[i, j]
        
        """ Delta(Margin) Function : MASK FOR WHICH NODES IN NODELIST BELONG TO DCS """
        gold_nodes_mask = np.array([False]*len(nodelist))
        gold_nodes_mask[list(nodelist_to_correct_mapping.values())] = True
        margin_f = lambda nodes_mask: np.sum(nodes_mask&(~gold_nodes_mask))**2
        
        """ FOR ALL POSSIBLE MST FROM THE COMPLETE GRAPH """
        WScalarMat = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, self.neuralnet)

        """ For each node - Find MST with that source"""
        min_STx = None # Min Energy spanning tree with worst margin with gold_STx
        min_marginalized_energy = np.inf
        
        # Generate random set of nodes from which mSTs are to be considered
        n_nodes = len(nodelist)
        selection_prob = 0.4
        select_flag = np.random.rand(n_nodes) < selection_prob
        # Fix if all zeros
        if np.sum(select_flag) == 0:
            select_flag[np.random.randint(n_nodes)] = 1
        for source in range(len(nodelist)):
            (mst_nodes, mst_adj_graph, mst_nodes_bool) = MST(nodelist, WScalarMat, conflicts_Dict, source) # T_X
            # print('.', end = '')
           
            marginalized_en = np.sum(WScalarMat[mst_adj_graph]) - margin_f(mst_nodes_bool)
            # Minimum marginalized spanning tree : Randomization applied
            # if marginalized_en < min_marginalized_energy and select_flag[source]:
            if marginalized_en < min_marginalized_energy:
                min_marginalized_energy = marginalized_en
                min_STx = mst_adj_graph
            # Energy diff should all be negative
            if _debug:
                print('Source: [{}], Node_Diff:{}, Max_Gold_En: {:.3f}, Energy: {:.3f}'.\
                      format(source, np.sum((~gold_nodes_mask)&mst_nodes_bool), energy_gold_max_ST,  np.sum(WScalarMat[mst_adj_graph])))

        
        """ Gradient Descent """
        # FOR MOST OFFENdING Y
        doBpp = False
        
        Total_Loss = energy_gold_max_ST - min_marginalized_energy
        if Total_Loss > 0:
            dLdOut = np.zeros_like(WScalarMat)
            dLdOut[max_st_adj_gold&(~min_STx)] = 1
            dLdOut[(~max_st_adj_gold)&min_STx] = -1
            if _debug:
                print('{}. '.format(sentenceObj.sent_id), end = '')
            self.neuralnet.Back_Prop(dLdOut, len(nodelist), featVMat, _debug)
        else:
            trainingStatus[sentenceObj.sent_id] = True
        if _debug:
            print("\nFileKey: %s, Loss: %6.3f" % (sentenceObj.sent_id, Total_Loss))

In [14]:
# trainer.neuralnet.U

In [24]:
trainer = None
def InitModule():
    global trainer
    trainer = Trainer()
InitModule()
trainingStatus = defaultdict(lambda: bool(False))
trainer.Load('outputs/train_t1646878175381/nnet.p')

In [16]:
"""
################################################################################################
################################################################################################
################################################################################################
"""
# if __name__ == '__main__':
# loaded_SKT = pickle.load(open('../Simultaneous_CompatSKT_10K.p', 'rb'))
# loaded_DCS = pickle.load(open('../Simultaneous_DCS_10K.p', 'rb'))
# main(loaded_SKT, loaded_DCS)
np.set_printoptions(suppress=False)

fn = TrainFiles[3].replace('.ds.bz2', '.p2')
print(fn)
for _ in range(1):
    trainer.Train(loaded_SKT[fn], loaded_DCS[fn], bz2_input_folder, ecl_mst=True, _debug = True)

# # fn = TestFiles[1].replace('.ds.bz2', '.p2')
# trainer.Test(loaded_SKT[fn], loaded_DCS[fn])
# print ("Not Implemented")

10025.p2
Source: [0], Node_Diff:2, Max_Gold_En: 88.222, Energy: 91.650
Source: [1], Node_Diff:2, Max_Gold_En: 88.222, Energy: 90.309
Source: [2], Node_Diff:2, Max_Gold_En: 88.222, Energy: 90.289
Source: [3], Node_Diff:3, Max_Gold_En: 88.222, Energy: 93.502
Source: [4], Node_Diff:3, Max_Gold_En: 88.222, Energy: 91.070
Source: [5], Node_Diff:2, Max_Gold_En: 88.222, Energy: 89.829
Source: [6], Node_Diff:2, Max_Gold_En: 88.222, Energy: 93.012
Source: [7], Node_Diff:2, Max_Gold_En: 88.222, Energy: 91.238
Source: [8], Node_Diff:2, Max_Gold_En: 88.222, Energy: 91.243
Source: [9], Node_Diff:2, Max_Gold_En: 88.222, Energy: 91.634
Source: [10], Node_Diff:3, Max_Gold_En: 88.222, Energy: 90.442
Source: [11], Node_Diff:2, Max_Gold_En: 88.222, Energy: 94.275
Source: [12], Node_Diff:2, Max_Gold_En: 88.222, Energy: 91.418
Source:  0
Found using ECL MinST: En: 90.0308394835845, Margin: 3
Found using ECL MaxST: En: 109.16272235150362, Margin: 4
Source:  1
Found using ECL MinST: En: 90.0308394835845, Mar

In [51]:
train = train_generator(loaded_SKT, loaded_DCS, bz2_input_folder, n_trainset = -1, filePerBatch = 10, iterationPerBatch = 5, _debug = False)

In [52]:
# Complete Training
# tips: try increasing iterations per batch
trainingStatus = defaultdict(lambda: bool(False)) # Reset it after 3 epochs of full-training set
train.__next__()

Weights Saved:  outputs/train_t1646878175381/nnet_i0.p
Batch:  0
['100066.ds.bz2', '100234.ds.bz2', '10025.ds.bz2', '100326.ds.bz2', '100335.ds.bz2', '100347.ds.bz2', '100514.ds.bz2', '100612.ds.bz2', '10074.ds.bz2', '100830.ds.bz2']
ITERATION IN 0
ITERATION IN 1
ITERATION IN 2
ITERATION IN 3
ITERATION IN 4
Avg. time taken by 1 file(1 iteration): 1.964
Weights Saved:  outputs/train_t1646878175381/nnet.p
Batch:  1
['100895.ds.bz2', '100954.ds.bz2', '100969.ds.bz2', '101051.ds.bz2', '101078.ds.bz2', '101096.ds.bz2', '101128.ds.bz2', '101150.ds.bz2', '101381.ds.bz2', '101393.ds.bz2']
ITERATION IN 0
ITERATION IN 1
ITERATION IN 2
ITERATION IN 3
ITERATION IN 4
Avg. time taken by 1 file(1 iteration): 1.443
Weights Saved:  outputs/train_t1646878175381/nnet.p
Batch:  2
['1014.ds.bz2', '101470.ds.bz2', '1015.ds.bz2', '101576.ds.bz2', '10168.ds.bz2', '101720.ds.bz2', '101824.ds.bz2', '101942.ds.bz2', '101973.ds.bz2', '102105.ds.bz2']
ITERATION IN 0
ITERATION IN 1
ITERATION IN 2
ITERATION IN 3
ITE

### Test Here

In [41]:
trainer = None
def InitModule():
    global trainer
    trainer = Trainer()
InitModule()
trainingStatus = defaultdict(lambda: bool(False))
trainer.Load('outputs/train_t1646878175381/nnet.p')

In [38]:
i = 0
fn = TrainFiles[0].replace('.ds.bz2', '.p2')
dsbz2_name = '../NewData/skt_dcs_DS.bz2_1L_bigram_10K/' + TrainFiles[0]
sentenceObj = loaded_SKT[fn]
dcsObj = loaded_DCS[fn]

In [43]:
results = trainer.Test(sentenceObj, dcsObj, dsbz2_name, _dump=True, _outFile='outputs/preds_1L_bigram.csv')

In [27]:
results

(6, 6, 7, 8)

### Checking bz2 Files

In [37]:
bz2_input_folder2 = '../NewData/skt_dcs_DS.bz2_1L_bigram_10K/'
dsbz2_name = '100066' + '.ds.bz2'
(nodelist_correct, conflicts_Dict_correct, featVMat_correct, nodelist_to_correct_mapping,\
    nodelist, conflicts_Dict, featVMat) = open_dsbz2(bz2_input_folder2 + dsbz2_name)

In [33]:
nodelist_correct

[WD_Node[C: 0, P: 0, tatas @(2) => tatas],
 WD_Node[C: 1, P: 0, mad @(32) => aham],
 WD_Node[C: 2, P: 0, api @(2) => api],
 WD_Node[C: 3, P: 0, kOravya @(49) => kOravya],
 WD_Node[C: 4, P: 0, roza @(3) => roza],
 WD_Node[C: 4, P: 4, vyAkula @(3) => vyAkula],
 WD_Node[C: 4, P: 11, locana @(29) => locanas]]

In [44]:
kx = 0
for node in nodelist:
    print(kx, node)
    kx += 1

0 WD_Node[C: 0, P: 0, yad @(31) => yat]
1 WD_Node[C: 0, P: 0, yat @(31) => yat]
2 WD_Node[C: 0, P: 0,  @(-10) => yat]
3 WD_Node[C: 0, P: 0,  @(-10) => yat]
4 WD_Node[C: 1, P: 0, brAhmaRa @(3) => brAhmaRa]
5 WD_Node[C: 1, P: 7, arTa @(49) => arTe]
6 WD_Node[C: 1, P: 7, ArTa @(49) => ArTe]
7 WD_Node[C: 2, P: 0, visfj @(-23) => visfjet]
8 WD_Node[C: 2, P: 2, sfj @(-23) => sfjet]
9 WD_Node[C: 3, P: 0, Atman @(72) => AtmAnam]
10 WD_Node[C: 3, P: 3, Ana @(31) => Anam]
11 WD_Node[C: 3, P: 3, an @(-41) => Anam]
12 WD_Node[C: 4, P: 0, api @(2) => api]
13 WD_Node[C: 5, P: 0, ca @(2) => ca]
14 WD_Node[C: 5, P: 1, Atmaja @(31) => Atmajam]
15 WD_Node[C: 5, P: 1, Atman @(3) => Atma]
16 WD_Node[C: 5, P: 5, ja @(31) => jam]


In [42]:
SeeDCS(loaded_DCS['154658.p2'])

DCS ANALYZE
---------------
yad brAhmaRArTe visfjed AtmAnam api cAtmajam   
[['yat'], ['brāhmaṇa', 'artha'], ['visṛj'], ['ātman'], ['api'], ['ca', 'ātmaja']]
Lemmas: ['yat', 'brAhmaRa', 'arTa', 'visfj', 'Atman', 'api', 'ca', 'Atmaja']
[['2'], ['3', '169'], ['-23'], ['69'], ['2'], ['2', '69']]



######   THE END
-----------------------------------------------------------------------